In [1]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Dict, List, Any, Optional
from langchain_core.messages import HumanMessage
import json
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import os
load_dotenv()
llm_key = os.getenv('llm_key')

In [3]:
class GraphState(TypedDict):
    user_request: str
    workflow_analysis: Dict[str, Any]  # AI-generated workflow understanding
    approval_chain: List[Dict[str, Any]]  # Dynamic approval hierarchy
    questions: Dict[str, Any]
    answers: Dict[str, Any]
    validation: Dict[str, Any]
    workflow: Dict[str, Any]
    form_schema: Dict[str, Any]
    excel_schema: Dict[str, Any]
    custom_rules: List[Dict[str, Any]]  # User-specific business rules
    sanity_check: Dict[str, Any]  # Validation results from sanity checker
    sanity_issues: List[str]  # Issues found during sanity check
    regeneration_count: int  # Track regeneration attempts
    iteration_count: int
    refinement_needed: bool
    conversation_history: List[str]  # Track clarifications

In [2]:
model=ChatOpenAI(model="openai/gpt-oss-20b:free",
                        api_key=llm_key,
                        base_url="https://openrouter.ai/api/v1")

In [ ]:
def regenerate_workflow_with_fixes(state: GraphState):
    """
    Regenerate workflow components based on sanity check issues
    """
    
    issues = state.get("sanity_issues", [])
    regeneration_count = state.get("regeneration_count", 0)
    
    print("\n" + "="*80)
    print(f"🔧 REGENERATING WORKFLOW (Attempt {regeneration_count + 1})")
    print("="*80)
    
    print("\n📋 Addressing the following issues:\n")
    for idx, issue in enumerate(issues, 1):
        print(f"   {idx}. {issue}")
    
    # Create a detailed prompt for AI to fix the issues
    prompt = f"""
    You are fixing a workflow generation that failed sanity checks.

    Original User Request:
    {state['user_request']}

    Issues Found:
    {chr(10).join(f"- {issue}" for issue in issues)}

    Current Workflow Analysis:
    {json.dumps(state.get('workflow_analysis', {}), indent=2)}

    Current Approval Chain:
    {json.dumps(state.get('approval_chain', []), indent=2)}

    Your task: Generate a CORRECTED workflow analysis that fixes ALL the issues above.
    Ensure:
    1. All required fields are present
    2. Approval chain logic is correct and sequential
    3. All approvers mentioned in user request are included
    4. Parallel approvals are properly configured if needed
    5. Rejection behaviors are defined
    6. All data structures are valid JSON

    Return ONLY the corrected workflow_analysis JSON.
    """
    workflow_analysis = state.get("workflow_analysis", {}).copy()
    approval_chain = state.get("approval_chain", []).copy()
    
    # Fix missing fields
    if not workflow_analysis.get("workflow_name"):
        workflow_analysis["workflow_name"] = "University Workflow"
    
    if not workflow_analysis.get("workflow_description"):
        workflow_analysis["workflow_description"] = "Automated approval workflow"
    
    # Ensure all required fields exist
    required_fields = [
        "workflow_name", "workflow_description", "stakeholders",
        "approval_chain", "data_to_collect", "notifications", "business_rules"
    ]
    
    for field in required_fields:
        if field not in workflow_analysis:
            workflow_analysis[field] = [] if field in ["stakeholders", "approval_chain", "data_to_collect", "notifications", "business_rules"] else ""
    
    # Fix approval chain levels
    if approval_chain:
        for idx, approver in enumerate(approval_chain):
            if "level" not in approver:
                approver["level"] = idx + 1
            if "approver_role" not in approver:
                approver["approver_role"] = f"Approver {idx + 1}"
            if "approver_type" not in approver:
                approver["approver_type"] = "single"
            if "rejection_behavior" not in approver:
                approver["rejection_behavior"] = "end_workflow"
            if "conditions" not in approver:
                approver["conditions"] = []
    
    print("\n✅ Applied fixes to workflow components")
    print(f"   Regeneration count: {regeneration_count + 1}")
    
    return {
        "workflow_analysis": workflow_analysis,
        "approval_chain": approval_chain,
        "regeneration_count": regeneration_count + 1
    }


In [11]:
def analyze_request(state: GraphState):
    """
    Deep AI analysis - understand EVERYTHING about the workflow from structured user input
    """
    
    prompt = f"""
You are an expert workflow architect for Microsoft Power Automate. Analyze this workflow request.

User Request:
{state['user_request']}

Previous Conversation: {state.get('conversation_history', [])}

The user has provided a structured request with:
1. Workflow Title/Purpose
2. Chain of Process (approval hierarchy)
3. Additional Requirements

CRITICAL REQUIREMENTS - ALWAYS ENFORCE:
- Microsoft Forms MUST be used to collect input from end users
- Excel MUST be used as the activity tracker for all requests, approvals, and rejections
- Microsoft Teams Approvals MUST be used for approval/rejection actions
- Outlook MUST be used for all email notifications
- Assume all Microsoft 365 credentials are configured in environment

Extract and return a comprehensive workflow analysis in VALID JSON format:

{{
  "workflow_name": "descriptive name from user input",
  "workflow_description": "what this workflow does",
  "workflow_purpose": "problem it solves",
  
  "stakeholders": [
    {{
      "role": "Student/Faculty/Staff/etc",
      "actions": ["submit request", "receive notifications"],
      "required_info": ["name", "email", "id"]
    }}
  ],
  
  "approval_chain": [
    {{
      "level": 1,
      "approver_role": "Mentor",
      "approver_type": "single|multiple",
      "source": "from_form|configured",
      "conditions": ["first in chain", "can add comments"],
      "rejection_behavior": "end_workflow|send_back|escalate|notify_director",
      "notification_rules": ["notify submitter", "notify director if rejected"]
    }},
    {{
      "level": 2,
      "approver_role": "Class Teacher",
      "approver_type": "single",
      "source": "from_form",
      "parallel_with_level": null,
      "conditions": ["must approve after mentor"],
      "rejection_behavior": "notify_director",
      "timeout_hours": 48
    }},
    {{
      "level": 3,
      "approver_role": "Director",
      "approver_type": "single",
      "source": "from_form",
      "requires_all_previous": true,
      "final_authority": true,
      "conditions": ["makes final decision", "receives all rejection notifications"],
      "rejection_behavior": "end_workflow"
    }}
  ],
  
  "approver_info_collection": [
    {{
      "role": "Mentor",
      "fields_needed": ["name", "email"],
      "collected_via": "form",
      "required": true
    }},
    {{
      "role": "Class Teacher",
      "fields_needed": ["name", "email"],
      "collected_via": "form",
      "required": true
    }}
  ],
  
  "required_documents": [
    {{
      "name": "Supporting Document",
      "format": [".pdf", ".docx"],
      "required": false,
      "max_size_mb": 10
    }}
  ],
  
  "data_to_collect": [
    {{
      "field_name": "student_name",
      "label": "Student Name",
      "type": "text",
      "required": true,
      "validation": "string"
    }},
    {{
      "field_name": "student_email",
      "label": "Student Email",
      "type": "email",
      "required": true,
      "validation": "email"
    }},
    {{
      "field_name": "mentor_name",
      "label": "Mentor Name",
      "type": "text",
      "required": true,
      "validation": "string",
      "purpose": "approver_identification"
    }},
    {{
      "field_name": "mentor_email",
      "label": "Mentor Email",
      "type": "email",
      "required": true,
      "validation": "email",
      "purpose": "approver_contact"
    }},
    {{
      "field_name": "class_teacher_name",
      "label": "Class Teacher Name",
      "type": "text",
      "required": true,
      "validation": "string",
      "purpose": "approver_identification"
    }},
    {{
      "field_name": "class_teacher_email",
      "label": "Class Teacher Email",
      "type": "email",
      "required": true,
      "validation": "email",
      "purpose": "approver_contact"
    }},
    {{
      "field_name": "director_email",
      "label": "Director Email",
      "type": "email",
      "required": true,
      "validation": "email",
      "purpose": "notification_contact"
    }},
    {{
      "field_name": "request_details",
      "label": "Request Details",
      "type": "textarea",
      "required": true,
      "validation": "minimum_50_characters"
    }}
  ],
  
  "notifications": [
    {{
      "trigger": "form_submitted",
      "recipients": ["submitter", "first_approver"],
      "platform": "Outlook",
      "template": "confirmation"
    }},
    {{
      "trigger": "approval_at_any_level",
      "recipients": ["submitter", "next_approver"],
      "platform": "Outlook",
      "template": "approval_progress"
    }},
    {{
      "trigger": "rejection_at_any_level",
      "recipients": ["submitter", "director"],
      "platform": "Outlook",
      "template": "rejection_notice",
      "special_rule": "director must be notified of all rejections"
    }},
    {{
      "trigger": "final_approval",
      "recipients": ["submitter", "all_approvers"],
      "platform": "Outlook",
      "template": "final_decision"
    }}
  ],
  
  "tracking_requirements": [
    "submission_timestamp",
    "current_status",
    "approver_names_and_emails",
    "approval_history",
    "comments_at_each_level",
    "time_spent_at_each_level",
    "rejection_notifications_sent",
    "final_decision"
  ],
  
  "business_rules": [
    "Extract all business rules from user's additional requirements",
    "Approvals must use Microsoft Teams Approvals",
    "All tracking in Excel",
    "All emails via Outlook",
    "Director receives notification on ANY rejection"
  ],
  
  "special_features": [
    "List any special requirements from user input",
    "Custom notification rules",
    "Special escalation procedures"
  ],
  
  "platform_requirements": {{
    "form_platform": "Microsoft Forms",
    "tracking_platform": "Excel Online (Business)",
    "approval_platform": "Microsoft Teams Approvals",
    "email_platform": "Office 365 Outlook",
    "storage_platform": "SharePoint"
  }}
}}

IMPORTANT PARSING RULES:
1. Parse the approval chain from the "Chain of process" line
2. Identify the sequence: "A to B to C" means A is level 1, B is level 2, C is level 3
3. Extract special notification rules from "Additional requirements"
4. If approver info needs to be collected from end user, add those fields to "data_to_collect"
5. Always use Microsoft Teams for approvals, not standalone approval actions
6. All activity tracking goes to Excel
7. All emails use Outlook

Return ONLY valid JSON. Be thorough and extract ALL implicit requirements.
"""
    
    # In production, replace with actual LLM call
    # response = model.invoke([HumanMessage(content=prompt)])
    # analysis = json.loads(response.content)
    
    # Parse the user request
    lines = state['user_request'].strip().split('\n')
    workflow_title = lines[0].strip() if len(lines) > 0 else "University Workflow"
    
    # Parse approval chain from second line
    approval_sequence = []
    if len(lines) > 1:
        chain_line = lines[1].strip().lower()
        # Parse "student to mentor to class teacher to director"
        parts = [p.strip() for p in chain_line.replace(' to ', '|').split('|')]
        approval_sequence = [p for p in parts if p and p != 'student']
    
    # Parse additional requirements
    additional_reqs = []
    if len(lines) > 2:
        additional_reqs = [line.strip() for line in lines[2:] if line.strip()]
    
    # Build approval chain
    approval_chain = []
    for idx, role in enumerate(approval_sequence, 1):
        role_title = role.title()
        
        # Check for special rejection rules in additional requirements
        rejection_behavior = "end_workflow"
        notification_rules = []
        
        for req in additional_reqs:
            if "reject" in req.lower() and "director" in req.lower():
                notification_rules.append("notify director on rejection")
                rejection_behavior = "notify_director"
        
        approval_chain.append({
            "level": idx,
            "approver_role": role_title,
            "approver_type": "single",
            "source": "from_form",
            "conditions": [f"Level {idx} approver", "Can approve/reject with comments"],
            "rejection_behavior": rejection_behavior,
            "notification_rules": notification_rules if notification_rules else [],
            "timeout_hours": 48
        })
    
    # Determine what data to collect
    data_fields = [
        {
            "field_name": "submitter_name",
            "label": "Your Full Name",
            "type": "text",
            "required": True,
            "validation": "string"
        },
        {
            "field_name": "submitter_email",
            "label": "Your Email",
            "type": "email",
            "required": True,
            "validation": "email"
        },
        {
            "field_name": "submitter_id",
            "label": "Your ID Number",
            "type": "text",
            "required": True,
            "validation": "string"
        },
        {
            "field_name": "request_details",
            "label": "Request Details",
            "type": "textarea",
            "required": True,
            "validation": "minimum_50_characters",
            "placeholder": "Provide detailed information about your request"
        }
    ]
    
    # Add fields for approver information if needed
    approver_info = []
    for role in approval_sequence:
        role_title = role.title().replace(" ", "_")
        
        # Check if user said approver info comes from form
        collect_from_form = any("provide" in req.lower() and role.lower() in req.lower() 
                               for req in additional_reqs)
        
        if collect_from_form or "provide" in state['user_request'].lower():
            data_fields.extend([
                {
                    "field_name": f"{role_title.lower()}_name",
                    "label": f"{role.title()} Name",
                    "type": "text",
                    "required": True,
                    "validation": "string",
                    "purpose": "approver_identification"
                },
                {
                    "field_name": f"{role_title.lower()}_email",
                    "label": f"{role.title()} Email",
                    "type": "email",
                    "required": True,
                    "validation": "email",
                    "purpose": "approver_contact"
                }
            ])
            
            approver_info.append({
                "role": role.title(),
                "fields_needed": ["name", "email"],
                "collected_via": "form",
                "required": True
            })
    
    # Build notifications based on requirements
    notifications = [
        {
            "trigger": "form_submitted",
            "recipients": ["submitter", "first_approver"],
            "platform": "Outlook",
            "template": "confirmation",
            "timing": "immediate"
        },
        {
            "trigger": "approval_at_any_level",
            "recipients": ["submitter", "next_approver"],
            "platform": "Outlook",
            "template": "approval_progress",
            "timing": "immediate"
        },
        {
            "trigger": "final_approval",
            "recipients": ["submitter", "all_approvers"],
            "platform": "Outlook",
            "template": "final_decision",
            "timing": "immediate"
        }
    ]
    
    # Add special rejection notification if required
    if any("reject" in req.lower() and "director" in req.lower() for req in additional_reqs):
        notifications.append({
            "trigger": "rejection_at_any_level",
            "recipients": ["submitter", "director"],
            "platform": "Outlook",
            "template": "rejection_notice",
            "special_rule": "director must be notified of all rejections",
            "timing": "immediate"
        })
    
    analysis = {
        "workflow_name": workflow_title,
        "workflow_description": f"Automated approval workflow for {workflow_title.lower()}",
        "workflow_purpose": f"Streamline the {workflow_title.lower()} process with automated approvals",
        
        "stakeholders": [
            {
                "role": "End User (Submitter)",
                "actions": ["submit request via form", "receive notifications", "track status"],
                "required_info": ["name", "email", "id"]
            }
        ] + [
            {
                "role": role.title(),
                "actions": ["review request", "approve/reject via Teams", "add comments"],
                "required_info": ["email"]
            }
            for role in approval_sequence
        ],
        
        "approval_chain": approval_chain,
        "approver_info_collection": approver_info,
        
        "required_documents": [
            {
                "name": "Supporting Documents",
                "format": [".pdf", ".docx", ".jpg", ".png"],
                "required": False,
                "max_size_mb": 10
            }
        ],
        
        "data_to_collect": data_fields,
        "notifications": notifications,
        
        "tracking_requirements": [
            "submission_id",
            "submission_timestamp",
            "submitter_details",
            "request_details",
            "current_status",
            "approver_names_and_emails",
            "approval_history_per_level",
            "comments_at_each_level",
            "time_at_each_level",
            "rejection_notifications_sent",
            "final_outcome"
        ],
        
        "business_rules": [
            f"Approval chain: {' → '.join([r.title() for r in approval_sequence])}",
            "All approvals via Microsoft Teams Approvals",
            "All tracking in Excel SharePoint table",
            "All emails via Outlook",
            "End user provides approver details via form"
        ] + [req for req in additional_reqs if req.strip()],
        
        "special_features": [
            "Dynamic approver assignment from form data",
            "Real-time Excel activity tracking",
            "Teams-based approval experience",
            "Comprehensive email notifications"
        ],
        
        "platform_requirements": {
            "form_platform": "Microsoft Forms",
            "tracking_platform": "Excel Online (Business)",
            "approval_platform": "Microsoft Teams - Approvals",
            "email_platform": "Office 365 Outlook",
            "storage_platform": "SharePoint"
        }
    }
    
    print("\n🔍 WORKFLOW ANALYSIS COMPLETE")
    print(f"   Workflow: {analysis['workflow_name']}")
    print(f"   Approval Chain: {' → '.join([a['approver_role'] for a in approval_chain])}")
    print(f"   Approval Levels: {len(approval_chain)}")
    print(f"   Data Fields: {len(data_fields)}")
    print(f"   Special Rules: {len([r for r in additional_reqs if r.strip()])}")
    print(f"   Platform: Microsoft 365 (Forms, Teams, Excel, Outlook)")
    
    return {
        "workflow_analysis": analysis,
        "approval_chain": analysis["approval_chain"],
        "conversation_history": state.get("conversation_history", []) + [f"Analyzed: {state['user_request']}"]
    }


In [12]:
def generate_questions(state: GraphState):
    """
    Generate questions dynamically based on AI workflow analysis
    """
    
    analysis = state.get("workflow_analysis", {})
    
    # AI already determined what data to collect
    data_fields = analysis.get("data_to_collect", [])
    documents = analysis.get("required_documents", [])
    
    questions = {
        "workflow_name": analysis.get("workflow_name", ""),
        "form_title": f"{analysis.get('workflow_name', 'Request Form')}",
        "form_description": analysis.get("workflow_description", ""),
        "required_fields": [],
        "optional_fields": [],
        "file_uploads": [],
        "auto_calculated_fields": []
    }
    
    # Convert AI analysis to form fields
    for field in data_fields:
        field_def = {
            "id": field["field_name"],
            "question": field["label"],
            "type": field["type"],
            "required": field["required"],
            "validation": field.get("validation", ""),
            "placeholder": f"Enter {field['label'].lower()}"
        }
        
        if field["type"] == "choice":
            field_def["choices"] = field.get("choices", [])
        
        if field.get("auto_calculate"):
            field_def["auto_calculate"] = field["auto_calculate"]
            questions["auto_calculated_fields"].append(field_def)
        
        if field["required"]:
            questions["required_fields"].append(field_def)
        else:
            questions["optional_fields"].append(field_def)
    
    # Add file uploads
    for doc in documents:
        questions["file_uploads"].append({
            "id": doc["name"].lower().replace(" ", "_"),
            "label": doc["name"],
            "description": doc.get("description", ""),
            "accepted_types": doc["format"],
            "max_size_mb": doc["max_size_mb"],
            "required": doc["required"]
        })
    
    print(f"\n📋 Generated {len(questions['required_fields'])} required fields")
    print(f"📎 {len(questions['file_uploads'])} document uploads configured")
    
    return {"questions": questions}

In [13]:
def collect_answers(state: GraphState):
    """
    Human-in-the-loop: Collect ONLY admin workflow configuration
    NOT end-user data - that goes in the form
    """
    
    analysis = state.get("workflow_analysis", {})
    approval_chain = state.get("approval_chain", [])
    
    print(f"\n{'='*80}")
    print(f"⚙️  ADMIN CONFIGURATION")
    print(f"{'='*80}")
    print(f"\n{analysis.get('workflow_description', '')}\n")
    
    print("📋 The Microsoft Form will collect:")
    print("   • End user details (name, email, ID)")
    print("   • Request details")
    print("   • Approver information (names and emails)")
    print("   • Supporting documents")
    
    print("\n🔧 Now configure ADMIN settings for the workflow:\n")
    
    answers = {}
    
    # SharePoint location
    print(f"{'─'*80}")
    print("📁 STORAGE CONFIGURATION")
    print(f"{'─'*80}\n")
    
    default_location = "Shared Documents/Workflow Tracking"
    location = input(f"SharePoint location for Excel tracker\n(default: {default_location}): ").strip()
    answers['sharepoint_location'] = location if location else default_location
    
    # Approval timeout settings
    print(f"\n{'─'*80}")
    print("⏰ APPROVAL TIMEOUT SETTINGS")
    print(f"{'─'*80}\n")
    
    default_timeout = 48
    timeout_input = input(f"Default timeout for approvals in hours (default: {default_timeout}): ").strip()
    answers['default_timeout'] = int(timeout_input) if timeout_input else default_timeout
    
    # Reminder settings
    reminder = input(f"Send reminders before timeout? (yes/no, default: yes): ").strip().lower()
    answers['send_reminders'] = reminder != 'no'
    
    if answers['send_reminders']:
        reminder_hours = input(f"Send reminder how many hours before timeout? (default: 24): ").strip()
        answers['reminder_hours'] = int(reminder_hours) if reminder_hours else 24
    
    # Admin notifications
    print(f"\n{'─'*80}")
    print("📧 ADMIN NOTIFICATION SETTINGS")
    print(f"{'─'*80}\n")
    
    admin_notif = input("Receive copy of all workflow notifications? (yes/no, default: no): ").strip().lower()
    answers['admin_notification_copy'] = admin_notif == 'yes'
    
    if answers['admin_notification_copy']:
        admin_email = input("Admin email address: ").strip()
        answers['admin_email'] = admin_email
    
    # Withdrawal settings
    print(f"\n{'─'*80}")
    print("🚫 REQUEST MANAGEMENT")
    print(f"{'─'*80}\n")
    
    allow_withdrawal = input("Allow submitters to withdraw requests before final approval? (yes/no, default: yes): ").strip().lower()
    answers['allow_withdrawal'] = allow_withdrawal != 'no'
    
    # Custom messages
    print(f"\n{'─'*80}")
    print("💬 CUSTOM MESSAGES (optional)")
    print(f"{'─'*80}\n")
    
    custom_approval_msg = input("Custom message for approval emails (press Enter to skip): ").strip()
    if custom_approval_msg:
        answers['custom_approval_message'] = custom_approval_msg
    
    custom_rejection_msg = input("Custom message for rejection emails (press Enter to skip): ").strip()
    if custom_rejection_msg:
        answers['custom_rejection_message'] = custom_rejection_msg
    
    # Teams settings
    print(f"\n{'─'*80}")
    print("👥 MICROSOFT TEAMS SETTINGS")
    print(f"{'─'*80}\n")
    
    print("✓ Approvals will be sent via Microsoft Teams")
    print("✓ Approvers will respond directly in Teams")
    print("✓ Comments and attachments supported")
    
    teams_channel = input("\nSend approval notifications to specific Teams channel? (channel name or press Enter to skip): ").strip()
    if teams_channel:
        answers['teams_channel'] = teams_channel
    
    print(f"\n{'='*80}")
    print("✅ Configuration complete!")
    print(f"{'='*80}\n")
    
    print("📋 CONFIGURATION SUMMARY:")
    print(f"   • SharePoint Location: {answers['sharepoint_location']}")
    print(f"   • Approval Timeout: {answers['default_timeout']} hours")
    print(f"   • Reminders: {'Enabled' if answers['send_reminders'] else 'Disabled'}")
    print(f"   • Admin Notifications: {'Enabled' if answers['admin_notification_copy'] else 'Disabled'}")
    print(f"   • Withdrawal Allowed: {'Yes' if answers['allow_withdrawal'] else 'No'}")
    print(f"   • Approval Platform: Microsoft Teams")
    print(f"   • Email Platform: Outlook")
    print(f"   • Tracking Platform: Excel (SharePoint)")
    
    return {"answers": answers}


In [14]:
def validate_answers(state: GraphState):
    """
    Validate collected admin configuration
    """
    
    answers = state.get("answers", {})
    
    missing = []
    invalid = []
    
    # Check SharePoint location
    if not answers.get('sharepoint_location'):
        missing.append("SharePoint location")
    
    # Check admin email if notification copy is enabled
    if answers.get('admin_notification_copy') and not answers.get('admin_email'):
        missing.append("Admin email address")
    elif answers.get('admin_notification_copy') and "@" not in answers.get('admin_email', ''):
        invalid.append("Admin email (invalid format)")
    
    # Check timeout is reasonable
    if answers.get('default_timeout'):
        timeout = answers['default_timeout']
        if timeout < 1 or timeout > 168:  # 1 hour to 1 week
            invalid.append(f"Default timeout ({timeout} hours) should be between 1 and 168 hours")
    
    is_valid = len(missing) == 0 and len(invalid) == 0
    
    validation_result = {
        "valid": is_valid,
        "missing": missing,
        "invalid": invalid
    }
    
    if not is_valid:
        print("\n❌ VALIDATION FAILED:")
        if missing:
            print(f"\n   Missing configuration: {', '.join(missing)}")
        if invalid:
            print(f"\n   Invalid configuration: {', '.join(invalid)}")
        print("\n   Please provide the missing/correct information.\n")
    else:
        print("\n✅ All configuration validated successfully!\n")
    
    return {
        "validation": validation_result,
        "refinement_needed": not is_valid
    }

In [15]:
def generate_form_schema(state: GraphState):
    """
    Generate Microsoft Forms JSON schema
    """
    
    questions = state.get("questions", {})
    
    form_schema = {
        "title": questions.get("form_title", "University Workflow Form"),
        "description": questions.get("form_description", ""),
        "settings": {
            "one_response_per_user": True,
            "allow_anonymous": False,
            "confirmation_message": "Your request has been submitted successfully!"
        },
        "questions": []
    }
    
    # Add required fields
    for idx, field in enumerate(questions.get("required_fields", []), 1):
        form_schema["questions"].append({
            "id": f"q{idx}",
            "type": field["type"],
            "title": field["question"],
            "required": field["required"],
            "validation": field.get("validation", ""),
            "placeholder": field.get("placeholder", "")
        })
    
    # Add optional fields
    for idx, field in enumerate(questions.get("optional_fields", []), len(form_schema["questions"]) + 1):
        form_schema["questions"].append({
            "id": f"q{idx}",
            "type": field["type"],
            "title": field["question"],
            "required": False
        })
    
    return {"form_schema": form_schema}


In [16]:
def generate_excel_schema(state: GraphState):
    """
    Generate Excel tracking schema based on workflow analysis
    """
    
    analysis = state.get("workflow_analysis", {})
    questions = state.get("questions", {})
    approval_chain = state.get("approval_chain", [])
    tracking_reqs = analysis.get("tracking_requirements", [])
    
    columns = [
        {"name": "SubmissionID", "type": "text", "description": "Unique identifier"},
        {"name": "SubmissionTimestamp", "type": "datetime", "description": "When request was submitted"},
        {"name": "CurrentStatus", "type": "choice", "choices": [
            "Submitted",
            "Pending Level 1",
            "Pending Level 2", 
            "Pending Final Approval",
            "Approved",
            "Rejected",
            "Withdrawn"
        ]}
    ]
    
    # Add columns for each form field
    for field in questions.get("required_fields", []):
        columns.append({
            "name": field["question"].replace(" ", "_"),
            "type": "text",
            "description": f"User input: {field['question']}"
        })
    
    # Add approval tracking columns for EACH level in the chain
    for level_info in approval_chain:
        level = level_info["level"]
        role = level_info["approver_role"].replace(" ", "_")
        
        columns.extend([
            {
                "name": f"{role}_Status",
                "type": "choice",
                "choices": ["Pending", "Approved", "Rejected", "Conditional"],
                "description": f"Decision by {level_info['approver_role']}"
            },
            {
                "name": f"{role}_Name",
                "type": "text",
                "description": f"Name of {level_info['approver_role']}"
            },
            {
                "name": f"{role}_Email",
                "type": "text",
                "description": f"Email of {level_info['approver_role']}"
            },
            {
                "name": f"{role}_Timestamp",
                "type": "datetime",
                "description": f"When {level_info['approver_role']} made decision"
            },
            {
                "name": f"{role}_Comments",
                "type": "text",
                "description": f"Comments from {level_info['approver_role']}"
            },
            {
                "name": f"{role}_TimeSpent_Hours",
                "type": "number",
                "description": f"Hours spent at {level_info['approver_role']} level"
            }
        ])
    
    # Add final tracking columns
    columns.extend([
        {"name": "FinalDecision", "type": "choice", "choices": ["Approved", "Rejected", "Withdrawn"]},
        {"name": "FinalDecisionDate", "type": "datetime"},
        {"name": "TotalProcessingTime_Hours", "type": "number"},
        {"name": "ApprovedBy", "type": "text", "description": "Final approver name"},
        {"name": "RejectedBy", "type": "text", "description": "Who rejected if applicable"},
        {"name": "RejectionReason", "type": "text"},
        {"name": "WithdrawalReason", "type": "text"},
        {"name": "NotificationsSent", "type": "text", "description": "List of all notifications"}
    ])
    
    # Add any custom tracking from business rules
    for req in tracking_reqs:
        if not any(req in col["name"] for col in columns):
            columns.append({
                "name": req.replace(" ", "_"),
                "type": "text",
                "description": f"Custom tracking: {req}"
            })
    
    excel_schema = {
        "table_name": f"{analysis.get('workflow_name', 'Workflow').replace(' ', '_')}_Tracker",
        "location": "SharePoint/Shared Documents/Workflow Tracking",
        "file_name": f"{analysis.get('workflow_name', 'Workflow').replace(' ', '_')}_Database.xlsx",
        "columns": columns,
        "auto_sync": True,
        "indexed_columns": ["SubmissionID", "CurrentStatus", "SubmissionTimestamp"],
        "views": [
            {
                "name": "Pending Approvals",
                "filter": "CurrentStatus <> 'Approved' AND CurrentStatus <> 'Rejected'"
            },
            {
                "name": "Approved Requests",
                "filter": "CurrentStatus = 'Approved'"
            },
            {
                "name": "This Week's Submissions",
                "filter": "SubmissionTimestamp >= TODAY() - 7"
            }
        ]
    }
    
    print(f"\n📊 Excel schema generated with {len(columns)} columns")
    print(f"   Includes tracking for {len(approval_chain)} approval levels")
    
    return {"excel_schema": excel_schema}


In [17]:
def generate_power_automate_workflow(state: GraphState):
    """
    Generate complete Power Automate workflow with dynamic approval chain
    """
    
    analysis = state.get("workflow_analysis", {})
    answers = state.get("answers", {})
    form_schema = state.get("form_schema", {})
    excel_schema = state.get("excel_schema", {})
    approval_chain = state.get("approval_chain", [])
    notifications = analysis.get("notifications", [])
    business_rules = analysis.get("business_rules", [])
    
    workflow = {
        "name": analysis.get("workflow_name", ""),
        "description": analysis.get("workflow_description", ""),
        "trigger": {
            "type": "Microsoft Forms",
            "operation": "When a new response is submitted",
            "form_id": "{FORM_ID_TO_BE_CONFIGURED}",
            "parameters": {
                "form_name": form_schema.get("title", "")
            }
        },
        "variables": [],
        "actions": [],
        "approval_flow": [],
        "notification_flow": [],
        "business_rules": business_rules,
        "configured_approvers": {}
    }
    
    # Store configured approver emails
    for level_info in approval_chain:
        role = level_info['approver_role']
        email_key = f"{role.lower().replace(' ', '_')}_email"
        workflow["configured_approvers"][role] = {
            "email": answers.get(email_key, ""),
            "timeout_hours": answers.get(f"{role.lower().replace(' ', '_')}_timeout", 48),
            "reminder_enabled": answers.get(f"{role.lower().replace(' ', '_')}_reminder", True)
        }
    
    # Initialize variables
    step_num = 1
    workflow["actions"].append({
        "step": step_num,
        "name": "Initialize Core Variables",
        "type": "Initialize variable",
        "variables": [
            {"name": "SubmissionID", "type": "String", "value": "@{guid()}"},
            {"name": "CurrentStatus", "type": "String", "value": "Submitted"},
            {"name": "SubmissionTimestamp", "type": "String", "value": "@{utcNow()}"},
            {"name": "ApprovalLevel", "type": "Integer", "value": 0},
            {"name": "AllApprovalsReceived", "type": "Boolean", "value": False},
            {"name": "WorkflowTerminated", "type": "Boolean", "value": False}
        ]
    })
    
    # Get form response details
    step_num += 1
    workflow["actions"].append({
        "step": step_num,
        "name": "Get Form Response Details",
        "type": "Microsoft Forms",
        "operation": "Get response details",
        "parameters": {
            "form_id": "{FORM_ID}",
            "response_id": "@{triggerBody()?['resourceData']?['id']}"
        },
        "note": "This captures all data submitted by the end user through the form"
    })
    
    # Create initial Excel row
    step_num += 1
    excel_row_values = {
        "SubmissionID": "@{variables('SubmissionID')}",
        "SubmissionTimestamp": "@{variables('SubmissionTimestamp')}",
        "CurrentStatus": "@{variables('CurrentStatus')}"
    }
    
    # Add all form fields to Excel (from end-user submissions)
    for field in form_schema.get("questions", [])[:5]:
        excel_row_values[field["title"].replace(" ", "_")] = f"@{{body('Get_Form_Response_Details')?['{field['id']}'] }}"
    
    workflow["actions"].append({
        "step": step_num,
        "name": "Create Tracking Record in Excel",
        "type": "Excel Online (Business)",
        "operation": "Add a row into a table",
        "parameters": {
            "location": "SharePoint",
            "document_library": excel_schema.get("location", ""),
            "file": excel_schema.get("file_name", ""),
            "table": excel_schema.get("table_name", ""),
            "values": excel_row_values
        },
        "note": "Creates a new row with end-user submitted data"
    })
    
    # Send initial confirmation to submitter (the END USER)
    step_num += 1
    submitter_email_field = next((f for f in form_schema.get("questions", []) if "email" in f.get("type", "").lower()), None)
    
    notification_recipients = [f"@{{body('Get_Form_Response_Details')?['{submitter_email_field['id'] if submitter_email_field else 'email'}']}}"]
    
    # Add admin to CC if configured
    if answers.get('admin_notification_copy'):
        cc_list = answers.get('admin_email', '')
    else:
        cc_list = ""
    
    workflow["actions"].append({
        "step": step_num,
        "name": "Send Submission Confirmation to End User",
        "type": "Office 365 Outlook",
        "operation": "Send an email (V2)",
        "parameters": {
            "to": f"@{{body('Get_Form_Response_Details')?['{submitter_email_field['id'] if submitter_email_field else 'email'}']}}",
            "cc": cc_list,
            "subject": f"✅ {analysis.get('workflow_name', '')} - Submission Received",
            "body": f"""
<div style="font-family: Arial, sans-serif; max-width: 600px;">
    <h2 style="color: #0078d4;">Submission Confirmed</h2>
    <p>Your request has been successfully submitted and is now being processed.</p>
    
    <div style="background-color: #f3f2f1; padding: 15px; border-radius: 5px; margin: 20px 0;">
        <strong>Submission ID:</strong> @{{variables('SubmissionID')}}<br>
        <strong>Status:</strong> @{{variables('CurrentStatus')}}<br>
        <strong>Submitted:</strong> @{{variables('SubmissionTimestamp')}}
    </div>
    
    <h3>Approval Process:</h3>
    <ol>
        {"".join(f"<li>{level['approver_role']}</li>" for level in approval_chain)}
    </ol>
    
    <p>You will receive notifications as your request progresses through each approval level.</p>
    
    {f"<p><em>Note: {answers.get('custom_approval_message', '')}</em></p>" if answers.get('custom_approval_message') else ''}
    
    <p style="color: #666; font-size: 12px;">This is an automated message. Please do not reply.</p>
</div>
"""
        },
        "note": "Confirms receipt to the end user who submitted the form"
    })
    
    # Build dynamic approval chain
    step_num += 1
    
    # Group parallel approvals
    approval_levels = {}
    for approval in approval_chain:
        level = approval["level"]
        if level not in approval_levels:
            approval_levels[level] = []
        approval_levels[level].append(approval)
    
    # Process each approval level
    for level, approvers in sorted(approval_levels.items()):
        
        # Check if previous level(s) approved
        if level > 1:
            step_num += 1
            workflow["approval_flow"].append({
                "step": step_num,
                "name": f"Check Previous Level Approvals",
                "type": "Condition",
                "expression": {
                    "and": [
                        {"equals": ["@variables('WorkflowTerminated')", False]},
                        {"equals": ["@variables('ApprovalLevel')", level - 1]}
                    ]
                },
                "if_yes": "continue",
                "if_no": "skip_to_end"
            })
        
        # Handle parallel approvals at same level
        if len(approvers) > 1:
            step_num += 1
            workflow["approval_flow"].append({
                "step": step_num,
                "name": f"Level {level} - Parallel Approval Setup",
                "type": "Scope",
                "description": f"Both {' and '.join([a['approver_role'] for a in approvers])} must approve",
                "parallel_execution": True
            })
        
        # Create approval action for each approver (CONFIGURED BY ADMIN)
        for approver in approvers:
            step_num += 1
            approver_role = approver["approver_role"]
            approver_email = workflow["configured_approvers"][approver_role]["email"]
            approver_timeout = workflow["configured_approvers"][approver_role]["timeout_hours"]
            
            workflow["approval_flow"].append({
                "step": step_num,
                "name": f"Approval Request - {approver_role}",
                "type": "Start and wait for an approval",
                "approval_type": "Approve/Reject - First to respond",
                "parameters": {
                    "title": f"🔔 {analysis.get('workflow_name', '')} - Approval Required",
                    "assigned_to": approver_email,
                    "details": f"""
<h3>Approval Request - Level {level}</h3>

<strong>Submitted by END USER via Form</strong><br>
<strong>Requester:</strong> @{{body('Get_Form_Response_Details')?['student_name']}}<br>
<strong>Submission ID:</strong> @{{variables('SubmissionID')}}<br>
<strong>Submitted:</strong> @{{variables('SubmissionTimestamp')}}

<h4>Request Details (from end user):</h4>
@{{body('Get_Form_Response_Details')?['reason']}}

<h4>Previous Approvals:</h4>
{_generate_previous_approvals_display(approval_chain, level)}

<strong>You have {approver_timeout} hours to respond</strong>

<p><em>Business Rules:</em></p>
<ul>
    {"".join(f"<li>{rule}</li>" for rule in business_rules[:3])}
</ul>
""",
                    "item_link": "@{concat('https://your-sharepoint.com/tracking/', variables('SubmissionID'))}",
                    "item_link_description": "View Full Request in System",
                    "enable_comments": True,
                    "enable_notifications": True,
                    "enable_reassignment": False
                },
                "timeout": {
                    "duration": f"PT{approver_timeout}H",
                    "action": "send_reminder" if workflow["configured_approvers"][approver_role]["reminder_enabled"] else "none"
                }
            })
            
            # Update Excel with approval decision
            step_num += 1
            workflow["approval_flow"].append({
                "step": step_num,
                "name": f"Update Excel - {approver_role} Decision",
                "type": "Excel Online (Business)",
                "operation": "Update a row",
                "parameters": {
                    "location": "SharePoint",
                    "document_library": excel_schema.get("location", ""),
                    "file": excel_schema.get("file_name", ""),
                    "table": excel_schema.get("table_name", ""),
                    "key_column": "SubmissionID",
                    "key_value": "@{variables('SubmissionID')}",
                    "values": {
                        f"{approver_role.replace(' ', '_')}_Status": f"@{{body('Approval_Request_-_{approver_role}')?['outcome']}}",
                        f"{approver_role.replace(' ', '_')}_Name": f"@{{body('Approval_Request_-_{approver_role}')?['responder']?['displayName']}}",
                        f"{approver_role.replace(' ', '_')}_Email": f"@{{body('Approval_Request_-_{approver_role}')?['responder']?['email']}}",
                        f"{approver_role.replace(' ', '_')}_Timestamp": "@{utcNow()}",
                        f"{approver_role.replace(' ', '_')}_Comments": f"@{{body('Approval_Request_-_{approver_role}')?['comments']}}"
                    }
                }
            })
            
            # Handle rejection
            step_num += 1
            rejection_behavior = approver.get("rejection_behavior", "end_workflow")
            
            rejection_message = answers.get('custom_rejection_message', 
                'Your request has been carefully reviewed but could not be approved at this time.')
            
            workflow["approval_flow"].append({
                "step": step_num,
                "name": f"Check {approver_role} Decision",
                "type": "Condition",
                "expression": {
                    "equals": [f"@{{body('Approval_Request_-_{approver_role}')?['outcome']}}", "Reject"]
                },
                "if_yes": {
                    "actions": [
                        {
                            "name": "Set Workflow Terminated",
                            "type": "Set variable",
                            "variable": "WorkflowTerminated",
                            "value": True
                        },
                        {
                            "name": "Update Status to Rejected",
                            "type": "Set variable",
                            "variable": "CurrentStatus",
                            "value": "Rejected"
                        },
                        {
                            "name": "Send Rejection Notification to End User",
                            "type": "Office 365 Outlook",
                            "operation": "Send an email (V2)",
                            "to": f"@{{body('Get_Form_Response_Details')?['{submitter_email_field['id'] if submitter_email_field else 'email'}']}}",
                            "cc": cc_list,
                            "subject": f"❌ {analysis.get('workflow_name', '')} - Request Not Approved",
                            "body": f"""
<div style="font-family: Arial, sans-serif;">
    <h2 style="color: #d13438;">Request Status Update</h2>
    <p>{rejection_message}</p>
    
    <div style="background-color: #fef0f0; padding: 15px; border-left: 4px solid #d13438; margin: 20px 0;">
        <strong>Reviewed by:</strong> {approver_role}<br>
        <strong>Date:</strong> @{{utcNow()}}<br>
        <strong>Comments:</strong><br>
        @{{body('Approval_Request_-_{approver_role}')?['comments']}}
    </div>
    
    <p>If you have questions, please contact {approver_role} at {approver_email}.</p>
    
    <p style="color: #666; font-size: 12px;">Submission ID: @{{variables('SubmissionID')}}</p>
</div>
"""
                        }
                    ]
                },
                "if_no": "continue"
            })
        
        # After all approvers at this level - check if all approved
        if len(approvers) > 1:
            step_num += 1
            workflow["approval_flow"].append({
                "step": step_num,
                "name": f"Verify All Level {level} Approvals",
                "type": "Condition",
                "description": f"Check that all {len(approvers)} approvers at level {level} approved",
                "expression": {
                    "and": [
                        {"equals": [f"@{{body('Approval_Request_-_{app['approver_role']}')?['outcome']}}", "Approve"]}
                        for app in approvers
                    ]
                },
                "if_yes": {
                    "actions": [
                        {
                            "name": f"Increment Approval Level to {level + 1}",
                            "type": "Set variable",
                            "variable": "ApprovalLevel",
                            "value": level
                        },
                        {
                            "name": "Update Status",
                            "type": "Set variable",
                            "variable": "CurrentStatus",
                            "value": f"Pending Level {level + 1}" if level < len(approval_levels) else "Final Approval Pending"
                        }
                    ]
                },
                "if_no": {
                    "actions": [
                        {
                            "name": "Terminate Workflow - Not All Approved",
                            "type": "Set variable",
                            "variable": "WorkflowTerminated",
                            "value": True
                        }
                    ]
                }
            })
    
    # Final approval received - notify END USER
    step_num += 1
    final_approver = approval_chain[-1]["approver_role"]
    
    workflow["actions"].append({
        "step": step_num,
        "name": "Final Approval Processing",
        "type": "Condition",
        "expression": {
            "and": [
                {"equals": [f"@{{body('Approval_Request_-_{final_approver}')?['outcome']}}", "Approve"]},
                {"equals": ["@variables('WorkflowTerminated')", False]}
            ]
        },
        "if_yes": {
            "actions": [
                {
                    "name": "Update Final Status in Excel",
                    "type": "Excel Online (Business)",
                    "operation": "Update a row",
                    "parameters": {
                        "table": excel_schema.get("table_name", ""),
                        "key_column": "SubmissionID",
                        "key_value": "@{variables('SubmissionID')}",
                        "values": {
                            "CurrentStatus": "Approved",
                            "FinalDecision": "Approved",
                            "FinalDecisionDate": "@{utcNow()}",
                            "ApprovedBy": f"@{{body('Approval_Request_-_{final_approver}')?['responder']?['displayName']}}"
                        }
                    }
                },
                {
                    "name": "Generate Approval Document for End User",
                    "type": "Word Online (Business)",
                    "operation": "Populate a template",
                    "parameters": {
                        "template": "{APPROVAL_LETTER_TEMPLATE}",
                        "data": {
                            "submission_id": "@{variables('SubmissionID')}",
                            "approval_date": "@{utcNow()}",
                            "all_form_data": "@{body('Get_Form_Response_Details')}"
                        }
                    }
                },
                {
                    "name": "Send Final Approval Email to End User",
                    "type": "Office 365 Outlook",
                    "operation": "Send an email (V2)",
                    "to": f"@{{body('Get_Form_Response_Details')?['{submitter_email_field['id'] if submitter_email_field else 'email'}']}}",
                    "cc": cc_list,
                    "subject": f"✅ {analysis.get('workflow_name', '')} - APPROVED",
                    "body": f"""
<div style="font-family: Arial, sans-serif;">
    <h2 style="color: #107c10;">Your Request Has Been Approved!</h2>
    <p>Congratulations! Your request has been fully approved by all required approvers.</p>
    
    <div style="background-color: #f0f9ff; padding: 20px; border-radius: 5px; margin: 20px 0;">
        <h3 style="margin-top: 0;">Approval Summary</h3>
        <strong>Final Approver:</strong> {final_approver}<br>
        <strong>Approval Date:</strong> @{{utcNow()}}<br>
        <strong>Submission ID:</strong> @{{variables('SubmissionID')}}
    </div>
    
    {f"<p><em>{answers.get('custom_approval_message', '')}</em></p>" if answers.get('custom_approval_message') else ''}
    
    <p>Please find your official approval document attached.</p>
    
    <p>Thank you!</p>
</div>
""",
                    "attachments": [
                        {
                            "name": "Approval_Document.pdf",
                            "content_bytes": "@{body('Generate_Approval_Document')}"
                        }
                    ]
                }
            ]
        }
    })
    
    # Add withdrawal capability if enabled
    if answers.get('allow_withdrawal'):
        workflow["actions"].append({
            "step": step_num + 1,
            "name": "Monitor for Withdrawal Requests",
            "type": "Recurrence",
            "description": "Allows end users to withdraw their requests before final approval",
            "note": "Implementation depends on your withdrawal mechanism (email, form, etc.)"
        })
    
    # Add notification handler
    for notification_config in notifications:
        workflow["notification_flow"].append({
            "trigger": notification_config["trigger"],
            "recipients": notification_config["recipients"],
            "template": notification_config["template"],
            "timing": notification_config.get("timing", "immediate")
        })
    
    print(f"\n⚙️ Generated complete Power Automate workflow")
    print(f"   Total actions: {len(workflow['actions']) + len(workflow['approval_flow'])}")
    print(f"   Approval levels: {len(approval_levels)}")
    print(f"   Configured approvers: {len(workflow['configured_approvers'])}")
    print(f"   Notifications: {len(workflow['notification_flow'])}")
    print(f"   Admin notification copy: {'Enabled' if answers.get('admin_notification_copy') else 'Disabled'}")
    
    return {"workflow": workflow}


In [18]:
def _generate_previous_approvals_display(approval_chain, current_level):
    """Helper to generate HTML for previous approvals"""
    if current_level == 1:
        return "<em>No previous approvals - This is the first level</em>"
    
    html = "<ul>"
    for approval in approval_chain[:current_level-1]:
        role = approval["approver_role"]
        html += f"""
        <li>
            <strong>{role}:</strong> 
            @{{{{body('Approval_Request_-_{role}')?['outcome']}}}} 
            by @{{{{body('Approval_Request_-_{role}')?['responder']?['displayName']}}}}
            <br><em>Comments: @{{{{body('Approval_Request_-_{role}')?['comments']}}}}</em>
        </li>
        """
    html += "</ul>"
    return html
    
    # Initialize variables
    step_num = 1
    workflow["actions"].append({
        "step": step_num,
        "name": "Initialize Core Variables",
        "type": "Initialize variable",
        "variables": [
            {"name": "SubmissionID", "type": "String", "value": "@{guid()}"},
            {"name": "CurrentStatus", "type": "String", "value": "Submitted"},
            {"name": "SubmissionTimestamp", "type": "String", "value": "@{utcNow()}"},
            {"name": "ApprovalLevel", "type": "Integer", "value": 0},
            {"name": "AllApprovalsReceived", "type": "Boolean", "value": False},
            {"name": "WorkflowTerminated", "type": "Boolean", "value": False}
        ]
    })
    
    # Get form response details
    step_num += 1
    workflow["actions"].append({
        "step": step_num,
        "name": "Get Form Response Details",
        "type": "Microsoft Forms",
        "operation": "Get response details",
        "parameters": {
            "form_id": "{FORM_ID}",
            "response_id": "@{triggerBody()?['resourceData']?['id']}"
        }
    })
    
    # Create initial Excel row
    step_num += 1
    excel_row_values = {
        "SubmissionID": "@{variables('SubmissionID')}",
        "SubmissionTimestamp": "@{variables('SubmissionTimestamp')}",
        "CurrentStatus": "@{variables('CurrentStatus')}"
    }
    
    # Add all form fields to Excel
    for field in form_schema.get("questions", [])[:5]:
        excel_row_values[field["title"].replace(" ", "_")] = f"@{{body('Get_Form_Response_Details')?['{field['id']}'] }}"
    
    workflow["actions"].append({
        "step": step_num,
        "name": "Create Tracking Record in Excel",
        "type": "Excel Online (Business)",
        "operation": "Add a row into a table",
        "parameters": {
            "location": "SharePoint",
            "document_library": excel_schema.get("location", ""),
            "file": excel_schema.get("file_name", ""),
            "table": excel_schema.get("table_name", ""),
            "values": excel_row_values
        }
    })
    
    # Send initial confirmation
    step_num += 1
    submitter_email_field = next((f for f in form_schema.get("questions", []) if "email" in f.get("type", "").lower()), None)
    
    workflow["actions"].append({
        "step": step_num,
        "name": "Send Submission Confirmation",
        "type": "Office 365 Outlook",
        "operation": "Send an email (V2)",
        "parameters": {
            "to": f"@{{body('Get_Form_Response_Details')?['{submitter_email_field['id'] if submitter_email_field else 'email'}']}}",
            "subject": f"✅ {analysis.get('workflow_name', '')} - Submission Received",
            "body": f"""
<div style="font-family: Arial, sans-serif; max-width: 600px;">
    <h2 style="color: #0078d4;">Submission Confirmed</h2>
    <p>Your request has been successfully submitted and is now being processed.</p>
    
    <div style="background-color: #f3f2f1; padding: 15px; border-radius: 5px; margin: 20px 0;">
        <strong>Submission ID:</strong> @{{variables('SubmissionID')}}<br>
        <strong>Status:</strong> @{{variables('CurrentStatus')}}<br>
        <strong>Submitted:</strong> @{{variables('SubmissionTimestamp')}}
    </div>
    
    <h3>Approval Process:</h3>
    <ol>
        {"".join(f"<li>{level['approver_role']}</li>" for level in approval_chain)}
    </ol>
    
    <p>You will receive notifications as your request progresses through each approval level.</p>
    
    <p style="color: #666; font-size: 12px;">This is an automated message. Please do not reply.</p>
</div>
"""
        }
    })
    
    # Build dynamic approval chain
    step_num += 1
    
    # Group parallel approvals
    approval_levels = {}
    for approval in approval_chain:
        level = approval["level"]
        if level not in approval_levels:
            approval_levels[level] = []
        approval_levels[level].append(approval)
    
    # Process each approval level
    for level, approvers in sorted(approval_levels.items()):
        
        # Check if previous level(s) approved
        if level > 1:
            step_num += 1
            workflow["approval_flow"].append({
                "step": step_num,
                "name": f"Check Previous Level Approvals",
                "type": "Condition",
                "expression": {
                    "and": [
                        {"equals": ["@variables('WorkflowTerminated')", False]},
                        {"equals": ["@variables('ApprovalLevel')", level - 1]}
                    ]
                },
                "if_yes": "continue",
                "if_no": "skip_to_end"
            })
        
        # Handle parallel approvals at same level
        if len(approvers) > 1:
            step_num += 1
            workflow["approval_flow"].append({
                "step": step_num,
                "name": f"Level {level} - Parallel Approval Setup",
                "type": "Scope",
                "description": f"Both {' and '.join([a['approver_role'] for a in approvers])} must approve",
                "parallel_execution": True
            })
        
        # Create approval action for each approver
        for approver in approvers:
            step_num += 1
            approver_role = approver["approver_role"]
            approver_field = approver_role.replace(" ", "_").lower()
            
            workflow["approval_flow"].append({
                "step": step_num,
                "name": f"Approval Request - {approver_role}",
                "type": "Start and wait for an approval",
                "approval_type": "Approve/Reject - First to respond",
                "parameters": {
                    "title": f"🔔 {analysis.get('workflow_name', '')} - Approval Required",
                    "assigned_to": f"{{{approver_role.upper().replace(' ', '_')}_EMAIL}}",
                    "details": f"""
<h3>Approval Request - Level {level}</h3>

<strong>Requester:</strong> @{{body('Get_Form_Response_Details')?['student_name']}}<br>
<strong>Submission ID:</strong> @{{variables('SubmissionID')}}<br>
<strong>Submitted:</strong> @{{variables('SubmissionTimestamp')}}

<h4>Request Details:</h4>
@{{body('Get_Form_Response_Details')?['reason']}}

<h4>Previous Approvals:</h4>
{self._generate_previous_approvals_display(approval_chain, level)}

<strong>Timeout:</strong> {approver.get('timeout_hours', 48)} hours

<p><em>Business Rules:</em></p>
<ul>
    {"".join(f"<li>{rule}</li>" for rule in business_rules[:3])}
</ul>
""",
                    "item_link": "@{concat('https://your-sharepoint.com/tracking/', variables('SubmissionID'))}",
                    "item_link_description": "View Full Request in System",
                    "enable_comments": True,
                    "enable_notifications": True,
                    "enable_reassignment": False
                },
                "timeout": {
                    "duration": f"PT{approver.get('timeout_hours', 48)}H",
                    "action": "send_reminder"
                }
            })
            
            # Update Excel with approval decision
            step_num += 1
            workflow["approval_flow"].append({
                "step": step_num,
                "name": f"Update Excel - {approver_role} Decision",
                "type": "Excel Online (Business)",
                "operation": "Update a row",
                "parameters": {
                    "location": "SharePoint",
                    "document_library": excel_schema.get("location", ""),
                    "file": excel_schema.get("file_name", ""),
                    "table": excel_schema.get("table_name", ""),
                    "key_column": "SubmissionID",
                    "key_value": "@{variables('SubmissionID')}",
                    "values": {
                        f"{approver_role.replace(' ', '_')}_Status": f"@{{body('Approval_Request_-_{approver_role}')?['outcome']}}",
                        f"{approver_role.replace(' ', '_')}_Name": f"@{{body('Approval_Request_-_{approver_role}')?['responder']?['displayName']}}",
                        f"{approver_role.replace(' ', '_')}_Email": f"@{{body('Approval_Request_-_{approver_role}')?['responder']?['email']}}",
                        f"{approver_role.replace(' ', '_')}_Timestamp": "@{utcNow()}",
                        f"{approver_role.replace(' ', '_')}_Comments": f"@{{body('Approval_Request_-_{approver_role}')?['comments']}}"
                    }
                }
            })
            
            # Handle rejection
            step_num += 1
            rejection_behavior = approver.get("rejection_behavior", "end_workflow")
            
            workflow["approval_flow"].append({
                "step": step_num,
                "name": f"Check {approver_role} Decision",
                "type": "Condition",
                "expression": {
                    "equals": [f"@{{body('Approval_Request_-_{approver_role}')?['outcome']}}", "Reject"]
                },
                "if_yes": {
                    "actions": [
                        {
                            "name": "Set Workflow Terminated",
                            "type": "Set variable",
                            "variable": "WorkflowTerminated",
                            "value": True
                        },
                        {
                            "name": "Update Status to Rejected",
                            "type": "Set variable",
                            "variable": "CurrentStatus",
                            "value": "Rejected"
                        },
                        {
                            "name": "Send Rejection Notification",
                            "type": "Office 365 Outlook",
                            "operation": "Send an email (V2)",
                            "to": f"@{{body('Get_Form_Response_Details')?['{submitter_email_field['id'] if submitter_email_field else 'email'}']}}",
                            "subject": f"❌ {analysis.get('workflow_name', '')} - Request Rejected",
                            "body": f"""
<div style="font-family: Arial, sans-serif;">
    <h2 style="color: #d13438;">Request Rejected</h2>
    <p>Your request has been rejected by {approver_role}.</p>
    
    <div style="background-color: #fef0f0; padding: 15px; border-left: 4px solid #d13438; margin: 20px 0;">
        <strong>Rejected by:</strong> @{{body('Approval_Request_-_{approver_role}')?['responder']?['displayName']}}<br>
        <strong>Date:</strong> @{{utcNow()}}<br>
        <strong>Comments:</strong><br>
        @{{body('Approval_Request_-_{approver_role}')?['comments']}}
    </div>
    
    <p>If you have questions, please contact {approver_role}.</p>
</div>
"""
                        }
                    ]
                },
                "if_no": "continue"
            })
        
        # After all approvers at this level - check if all approved
        if len(approvers) > 1:
            step_num += 1
            workflow["approval_flow"].append({
                "step": step_num,
                "name": f"Verify All Level {level} Approvals",
                "type": "Condition",
                "description": f"Check that all {len(approvers)} approvers at level {level} approved",
                "expression": {
                    "and": [
                        {"equals": [f"@{{body('Approval_Request_-_{app['approver_role']}')?['outcome']}}", "Approve"]}
                        for app in approvers
                    ]
                },
                "if_yes": {
                    "actions": [
                        {
                            "name": f"Increment Approval Level to {level + 1}",
                            "type": "Set variable",
                            "variable": "ApprovalLevel",
                            "value": level
                        },
                        {
                            "name": "Update Status",
                            "type": "Set variable",
                            "variable": "CurrentStatus",
                            "value": f"Pending Level {level + 1}" if level < len(approval_levels) else "Final Approval Pending"
                        }
                    ]
                },
                "if_no": {
                    "actions": [
                        {
                            "name": "Terminate Workflow - Not All Approved",
                            "type": "Set variable",
                            "variable": "WorkflowTerminated",
                            "value": True
                        }
                    ]
                }
            })
    
    # Final approval received
    step_num += 1
    final_approver = approval_chain[-1]["approver_role"]
    
    workflow["actions"].append({
        "step": step_num,
        "name": "Final Approval Processing",
        "type": "Condition",
        "expression": {
            "and": [
                {"equals": [f"@{{body('Approval_Request_-_{final_approver}')?['outcome']}}", "Approve"]},
                {"equals": ["@variables('WorkflowTerminated')", False]}
            ]
        },
        "if_yes": {
            "actions": [
                {
                    "name": "Update Final Status",
                    "type": "Excel Online (Business)",
                    "operation": "Update a row",
                    "parameters": {
                        "table": excel_schema.get("table_name", ""),
                        "key_column": "SubmissionID",
                        "key_value": "@{variables('SubmissionID')}",
                        "values": {
                            "CurrentStatus": "Approved",
                            "FinalDecision": "Approved",
                            "FinalDecisionDate": "@{utcNow()}",
                            "ApprovedBy": f"@{{body('Approval_Request_-_{final_approver}')?['responder']?['displayName']}}"
                        }
                    }
                },
                {
                    "name": "Generate Approval Document",
                    "type": "Word Online (Business)",
                    "operation": "Populate a template",
                    "parameters": {
                        "template": "{APPROVAL_LETTER_TEMPLATE}",
                        "data": {
                            "submission_id": "@{variables('SubmissionID')}",
                            "approval_date": "@{utcNow()}",
                            "all_form_data": "@{body('Get_Form_Response_Details')}"
                        }
                    }
                },
                {
                    "name": "Send Final Approval Email",
                    "type": "Office 365 Outlook",
                    "operation": "Send an email (V2)",
                    "to": f"@{{body('Get_Form_Response_Details')?['{submitter_email_field['id'] if submitter_email_field else 'email'}']}}",
                    "cc": f"{{{', '.join([a['approver_role'].upper().replace(' ', '_') + '_EMAIL' for a in approval_chain])}}}",
                    "subject": f"✅ {analysis.get('workflow_name', '')} - APPROVED",
                    "body": """
<div style="font-family: Arial, sans-serif;">
    <h2 style="color: #107c10;">Request Approved!</h2>
    <p>Congratulations! Your request has been fully approved.</p>
    
    <div style="background-color: #f0f9ff; padding: 20px; border-radius: 5px; margin: 20px 0;">
        <h3 style="margin-top: 0;">Approval Summary</h3>
        <strong>Final Approver:</strong> @{body('Approval_Request_-_Director')?['responder']?['displayName']}<br>
        <strong>Approval Date:</strong> @{utcNow()}<br>
        <strong>Submission ID:</strong> @{variables('SubmissionID')}
    </div>
    
    <p>Please find your official approval document attached.</p>
    
    <p>Thank you!</p>
</div>
""",
                    "attachments": [
                        {
                            "name": "Approval_Document.pdf",
                            "content_bytes": "@{body('Generate_Approval_Document')}"
                        }
                    ]
                }
            ]
        }
    })
    
    # Add notification handler
    for notification_config in notifications:
        workflow["notification_flow"].append({
            "trigger": notification_config["trigger"],
            "recipients": notification_config["recipients"],
            "template": notification_config["template"],
            "timing": notification_config.get("timing", "immediate")
        })
    
    print(f"\n⚙️ Generated complete Power Automate workflow")
    print(f"   Total actions: {len(workflow['actions']) + len(workflow['approval_flow'])}")
    print(f"   Approval levels: {len(approval_levels)}")
    print(f"   Notifications configured: {len(workflow['notification_flow'])}")
    
    return {"workflow": workflow}


In [19]:
def _generate_previous_approvals_display(self, approval_chain, current_level):
    """Helper to generate HTML for previous approvals"""
    if current_level == 1:
        return "<em>No previous approvals</em>"
    
    html = "<ul>"
    for approval in approval_chain[:current_level-1]:
        role = approval["approver_role"]
        html += f"""
        <li>
            <strong>{role}:</strong> 
            @{{{{body('Approval_Request_-_{role}')?['outcome']}}}} 
            by @{{{{body('Approval_Request_-_{role}')?['responder']?['displayName']}}}}
            <br><em>Comments: @{{{{body('Approval_Request_-_{role}')?['comments']}}}}</em>
        </li>
        """
    html += "</ul>"
    return html


In [20]:
def sanity_check_workflow(state: GraphState):
    """
    Comprehensive sanity check to validate the generated workflow
    Checks format, syntax, completeness, and alignment with user request
    """
    
    print("\n" + "="*80)
    print("🔍 RUNNING SANITY CHECK")
    print("="*80)
    
    issues = []
    warnings = []
    
    user_request = state.get("user_request", "")
    workflow_analysis = state.get("workflow_analysis", {})
    approval_chain = state.get("approval_chain", [])
    form_schema = state.get("form_schema", {})
    excel_schema = state.get("excel_schema", {})
    workflow = state.get("workflow", {})
    answers = state.get("answers", {})
    
    print("\n📋 Checking workflow components...\n")
    
    # ============================================================================
    # 1. CHECK WORKFLOW ANALYSIS
    # ============================================================================
    print("1️⃣  Validating Workflow Analysis...")
    
    required_analysis_fields = [
        "workflow_name", "workflow_description", "stakeholders", 
        "approval_chain", "data_to_collect", "notifications", 
        "business_rules"
    ]
    
    for field in required_analysis_fields:
        if field not in workflow_analysis or not workflow_analysis[field]:
            issues.append(f"Workflow analysis missing required field: {field}")
    
    # Check if stakeholders are properly defined
    if "stakeholders" in workflow_analysis:
        for stakeholder in workflow_analysis.get("stakeholders", []):
            if "role" not in stakeholder:
                issues.append(f"Stakeholder missing 'role' field: {stakeholder}")
    
    print(f"   ✓ Analysis structure validated")
    
    # ============================================================================
    # 2. CHECK APPROVAL CHAIN LOGIC
    # ============================================================================
    print("2️⃣  Validating Approval Chain Logic...")
    
    if not approval_chain or len(approval_chain) == 0:
        issues.append("Approval chain is empty - no approvers defined")
    else:
        # Check approval levels are sequential
        levels = [a["level"] for a in approval_chain]
        if levels != sorted(levels):
            issues.append(f"Approval levels are not sequential: {levels}")
        
        # Check for gaps in levels
        expected_levels = list(range(1, max(levels) + 1))
        if sorted(set(levels)) != expected_levels:
            issues.append(f"Approval chain has gaps in levels: {sorted(set(levels))}")
        
        # Check each approver has required fields
        for approver in approval_chain:
            required_approver_fields = ["level", "approver_role", "approver_type", "rejection_behavior"]
            for field in required_approver_fields:
                if field not in approver:
                    issues.append(f"Approver at level {approver.get('level', '?')} missing field: {field}")
            
            # Check if email is configured
            role = approver.get("approver_role", "")
            email_key = f"{role.lower().replace(' ', '_')}_email"
            if email_key not in answers or not answers[email_key]:
                issues.append(f"No email configured for approver: {role}")
            elif "@" not in answers[email_key]:
                issues.append(f"Invalid email format for approver: {role}")
        
        # Check for parallel approvals logic
        parallel_levels = {}
        for approver in approval_chain:
            level = approver["level"]
            if level not in parallel_levels:
                parallel_levels[level] = []
            parallel_levels[level].append(approver["approver_role"])
        
        for level, roles in parallel_levels.items():
            if len(roles) > 1:
                print(f"   ℹ️  Level {level} has parallel approvals: {', '.join(roles)}")
    
    print(f"   ✓ Approval chain logic validated")
    
    # ============================================================================
    # 3. CHECK FORM SCHEMA
    # ============================================================================
    print("3️⃣  Validating Microsoft Forms Schema...")
    
    if not form_schema:
        issues.append("Form schema is empty")
    else:
        # Check required form fields
        if "form_title" not in form_schema or not form_schema["form_title"]:
            issues.append("Form schema missing 'form_title'")
        
        if "required_fields" not in form_schema:
            issues.append("Form schema missing 'required_fields'")
        else:
            required_fields = form_schema["required_fields"]
            if len(required_fields) == 0:
                warnings.append("Form has no required fields - is this intentional?")
            
            # Check each field has proper structure
            for field in required_fields:
                if "id" not in field or "question" not in field or "type" not in field:
                    issues.append(f"Form field missing required properties: {field}")
                
                # Validate field types
                valid_types = ["text", "email", "phone", "number", "date", "textarea", "choice"]
                if field.get("type") not in valid_types:
                    issues.append(f"Invalid field type '{field.get('type')}' for field '{field.get('question')}'")
                
                # Check choice fields have choices
                if field.get("type") == "choice" and "choices" not in field:
                    issues.append(f"Choice field '{field.get('question')}' missing choices")
            
            # Check for email field (required for notifications)
            has_email = any(f.get("type") == "email" for f in required_fields)
            if not has_email:
                issues.append("Form must have at least one email field for notifications")
    
    print(f"   ✓ Form schema validated")
    
    # ============================================================================
    # 4. CHECK EXCEL SCHEMA
    # ============================================================================
    print("4️⃣  Validating Excel Tracking Schema...")
    
    if not excel_schema:
        issues.append("Excel schema is empty")
    else:
        # Check required Excel fields
        required_excel_fields = ["table_name", "location", "columns"]
        for field in required_excel_fields:
            if field not in excel_schema:
                issues.append(f"Excel schema missing required field: {field}")
        
        if "columns" in excel_schema:
            columns = excel_schema["columns"]
            
            # Check for essential tracking columns
            column_names = [c["name"] for c in columns]
            essential_columns = ["SubmissionID", "SubmissionTimestamp", "CurrentStatus"]
            
            for col in essential_columns:
                if col not in column_names:
                    issues.append(f"Excel schema missing essential column: {col}")
            
            # Check each approver has tracking columns
            for approver in approval_chain:
                role = approver["approver_role"].replace(" ", "_")
                expected_cols = [
                    f"{role}_Status",
                    f"{role}_Timestamp",
                    f"{role}_Comments"
                ]
                
                for col in expected_cols:
                    if col not in column_names:
                        issues.append(f"Excel missing tracking column for {approver['approver_role']}: {col}")
            
            # Validate column structure
            for col in columns:
                if "name" not in col or "type" not in col:
                    issues.append(f"Excel column missing required properties: {col}")
    
    print(f"   ✓ Excel schema validated")
    
    # ============================================================================
    # 5. CHECK POWER AUTOMATE WORKFLOW
    # ============================================================================
    print("5️⃣  Validating Power Automate Workflow...")
    
    if not workflow:
        issues.append("Power Automate workflow is empty")
    else:
        # Check workflow structure
        required_workflow_fields = ["name", "description", "trigger", "actions", "approval_flow"]
        for field in required_workflow_fields:
            if field not in workflow:
                issues.append(f"Workflow missing required field: {field}")
        
        # Check trigger configuration
        if "trigger" in workflow:
            trigger = workflow["trigger"]
            if trigger.get("type") != "Microsoft Forms":
                issues.append(f"Invalid trigger type: {trigger.get('type')} (should be 'Microsoft Forms')")
        
        # Check actions
        if "actions" in workflow:
            actions = workflow["actions"]
            if len(actions) == 0:
                issues.append("Workflow has no actions defined")
            
            # Check for essential actions
            action_names = [a.get("name", "") for a in actions]
            essential_actions = [
                "Initialize Core Variables",
                "Get Form Response Details",
                "Create Tracking Record in Excel"
            ]
            
            for essential in essential_actions:
                if not any(essential in name for name in action_names):
                    issues.append(f"Workflow missing essential action: {essential}")
        
        # Check approval flow
        if "approval_flow" in workflow:
            approval_flow = workflow["approval_flow"]
            
            # Count approval requests
            approval_requests = [a for a in approval_flow if "Approval Request" in a.get("name", "")]
            
            if len(approval_requests) != len(approval_chain):
                warnings.append(f"Approval flow has {len(approval_requests)} requests but approval chain has {len(approval_chain)} approvers")
            
            # Check each approval has update action
            for request in approval_requests:
                request_name = request.get("name", "")
                role = request_name.replace("Approval Request - ", "")
                
                # Look for corresponding update action
                has_update = any(f"Update Excel - {role}" in a.get("name", "") for a in approval_flow)
                if not has_update:
                    issues.append(f"Missing Excel update action for approval: {role}")
        
        # Check configured approvers
        if "configured_approvers" in workflow:
            for role, config in workflow["configured_approvers"].items():
                if "email" not in config or not config["email"]:
                    issues.append(f"Approver '{role}' has no configured email")
                elif "@" not in config["email"]:
                    issues.append(f"Approver '{role}' has invalid email format: {config['email']}")
    
    print(f"   ✓ Workflow structure validated")
    
    # ============================================================================
    # 6. CHECK ALIGNMENT WITH USER REQUEST
    # ============================================================================
    print("6️⃣  Validating Alignment with User Request...")
    
    # Extract key requirements from user request
    user_request_lower = user_request.lower()
    
    # Check if key roles mentioned in request are in approval chain
    common_roles = ["teacher", "mentor", "director", "supervisor", "head", "dean", "manager"]
    mentioned_roles = [role for role in common_roles if role in user_request_lower]
    
    approval_roles = [a["approver_role"].lower() for a in approval_chain]
    
    for role in mentioned_roles:
        if not any(role in approval_role for approval_role in approval_roles):
            warnings.append(f"User mentioned '{role}' but it's not clearly in approval chain: {approval_roles}")
    
    # Check for parallel approval requirements
    if "both" in user_request_lower or "and" in user_request_lower:
        # Check if we have parallel approvals
        has_parallel = any(len([a for a in approval_chain if a["level"] == level]) > 1 
                          for level in set(a["level"] for a in approval_chain))
        if not has_parallel:
            warnings.append("User request suggests parallel approvals ('both'/'and') but no parallel levels detected")
    
    # Check for rejection handling
    if "reject" in user_request_lower:
        rejection_behaviors = [a.get("rejection_behavior", "") for a in approval_chain]
        if not rejection_behaviors:
            warnings.append("User mentioned rejection but no rejection behavior configured")
    
    print(f"   ✓ Alignment checked")
    
    # ============================================================================
    # 7. VALIDATE JSON STRUCTURE
    # ============================================================================
    print("7️⃣  Validating JSON Structure...")
    
    try:
        # Try to serialize all components to JSON
        json.dumps(workflow_analysis)
        json.dumps(form_schema)
        json.dumps(excel_schema)
        json.dumps(workflow)
        print(f"   ✓ All components are valid JSON")
    except Exception as e:
        issues.append(f"JSON serialization error: {str(e)}")
    
    # ============================================================================
    # SANITY CHECK RESULTS
    # ============================================================================
    print("\n" + "="*80)
    print("📊 SANITY CHECK RESULTS")
    print("="*80)
    
    if len(issues) == 0 and len(warnings) == 0:
        print("\n✅ ALL CHECKS PASSED!")
        print("   The workflow is ready for deployment to Power Automate.")
        
        sanity_result = {
            "passed": True,
            "issues": [],
            "warnings": [],
            "message": "All validation checks passed successfully"
        }
    
    elif len(issues) == 0 and len(warnings) > 0:
        print(f"\n⚠️  PASSED WITH {len(warnings)} WARNING(S):")
        for idx, warning in enumerate(warnings, 1):
            print(f"   {idx}. {warning}")
        
        print("\n   The workflow is functional but you may want to review the warnings.")
        
        sanity_result = {
            "passed": True,
            "issues": [],
            "warnings": warnings,
            "message": "Passed with warnings"
        }
    
    else:
        print(f"\n❌ SANITY CHECK FAILED - {len(issues)} ISSUE(S) FOUND:")
        for idx, issue in enumerate(issues, 1):
            print(f"   {idx}. {issue}")
        
        if warnings:
            print(f"\n⚠️  Additional {len(warnings)} warning(s):")
            for idx, warning in enumerate(warnings, 1):
                print(f"   {idx}. {warning}")
        
        print("\n   The workflow will be regenerated to fix these issues.")
        
        sanity_result = {
            "passed": False,
            "issues": issues,
            "warnings": warnings,
            "message": "Validation failed - regeneration required"
        }
    
    print("="*80 + "\n")
    
    return {
        "sanity_check": sanity_result,
        "sanity_issues": issues
    }
    """
    Display the generated workflow to the admin user
    """
    
    workflow = state.get("workflow", {})
    form_schema = state.get("form_schema", {})
    excel_schema = state.get("excel_schema", {})
    analysis = state.get("workflow_analysis", {})
    approval_chain = state.get("approval_chain", [])
    
    print("\n" + "="*80)
    print("✅ WORKFLOW GENERATION COMPLETE")
    print("="*80)
    
    print(f"\n📋 WORKFLOW: {workflow.get('name', '')}")
    print(f"📝 {workflow.get('description', '')}")
    
    print("\n" + "-"*80)
    print("🎯 WORKFLOW COMPONENTS:")
    print("-"*80)
    
    print(f"\n1️⃣  MICROSOFT FORM")
    print(f"    Name: {form_schema.get('title', '')}")
    print(f"    Fields: {len(form_schema.get('questions', []))} questions")
    print(f"    Required: {sum(1 for q in form_schema.get('questions', []) if q.get('required'))}")
    print(f"    File uploads: {len([q for q in form_schema.get('questions', []) if 'file' in q.get('type', '')])}")
    
    print(f"\n2️⃣  EXCEL TRACKER") 
    print(f"    Table: {excel_schema.get('table_name', '')}")
    print(f"    Location: {excel_schema.get('location', '')}")
    print(f"    Columns: {len(excel_schema.get('columns', []))}")
    print(f"    Views: {len(excel_schema.get('views', []))}")
    
    print(f"\n3️⃣  APPROVAL CHAIN ({len(approval_chain)} levels)")
    for level_info in approval_chain:
        print(f"    Level {level_info['level']}: {level_info['approver_role']}")
        print(f"       • Type: {level_info['approver_type']}")
        print(f"       • Timeout: {level_info.get('timeout_hours', 48)} hours")
        print(f"       • On Rejection: {level_info['rejection_behavior']}")
    
    print(f"\n4️⃣  NOTIFICATIONS")
    notif_count = len(analysis.get('notifications', []))
    print(f"    Configured: {notif_count} notification triggers")
    for notif in analysis.get('notifications', [])[:3]:
        print(f"    • {notif['trigger']}: {', '.join(notif['recipients'])}")
    
    print(f"\n5️⃣  BUSINESS RULES")
    rules = analysis.get('business_rules', [])
    print(f"    Total Rules: {len(rules)}")
    for idx, rule in enumerate(rules[:5], 1):
        print(f"    {idx}. {rule}")
    
    print("\n" + "-"*80)
    print("📊 WORKFLOW STATISTICS:")
    print("-"*80)
    print(f"  Total Actions: {len(workflow.get('actions', []))}")
    print(f"  Approval Steps: {len(workflow.get('approval_flow', []))}")
    print(f"  Variables: {len(workflow.get('variables', []))}")
    print(f"  Conditional Logic: {sum(1 for a in workflow.get('approval_flow', []) if a.get('type') == 'Condition')}")
    
    print("\n" + "-"*80)
    print("💾 EXPORT OPTIONS:")
    print("-"*80)
    print("  1. Complete Package (JSON) - All components")
    print("  2. Power Automate Workflow Only")
    print("  3. Microsoft Forms Schema Only")
    print("  4. Excel Tracking Schema Only")
    print("  5. Implementation Guide (Human Readable)")
    print("  6. All Separate Files")
    
    choice = input("\n📤 Select export option (1-6): ").strip()
    
    print("\n" + "="*80)
    
    if choice == "1":
        package = {
            "metadata": {
                "workflow_name": analysis.get("workflow_name"),
                "generated_at": "2024-12-02T10:00:00Z",
                "version": "1.0"
            },
            "workflow_analysis": analysis,
            "power_automate_workflow": workflow,
            "microsoft_forms_schema": form_schema,
            "excel_tracking_schema": excel_schema,
            "implementation_notes": {
                "step_1": "Import Microsoft Forms schema",
                "step_2": "Create Excel table in SharePoint using schema",
                "step_3": "Import Power Automate workflow",
                "step_4": "Configure email addresses for approvers",
                "step_5": "Test with sample submission",
                "step_6": "Deploy to production"
            }
        }
        print(json.dumps(package, indent=2))
        
    elif choice == "2":
        print(json.dumps(workflow, indent=2))
        
    elif choice == "3":
        print(json.dumps(form_schema, indent=2))
        
    elif choice == "4":
        print(json.dumps(excel_schema, indent=2))
        
    elif choice == "5":
        print(_generate_implementation_guide(analysis, workflow, form_schema, excel_schema, approval_chain))
        
    elif choice == "6":
        print("\n📁 Generating separate files...\n")
        print("=" *80)
        print("FILE 1: power_automate_workflow.json")
        print("="*80)
        print(json.dumps(workflow, indent=2))
        print("\n" + "="*80)
        print("FILE 2: microsoft_forms_schema.json")
        print("="*80)
        print(json.dumps(form_schema, indent=2))
        print("\n" + "="*80)
        print("FILE 3: excel_tracking_schema.json")
        print("="*80)
        print(json.dumps(excel_schema, indent=2))
        print("\n" + "="*80)
        print("FILE 4: workflow_analysis.json")
        print("="*80)
        print(json.dumps(analysis, indent=2))
    
    return state

In [21]:
def _generate_implementation_guide(analysis, workflow, form_schema, excel_schema, approval_chain):
    """Generate human-readable implementation guide"""
    
    guide = f"""
{'='*80}
IMPLEMENTATION GUIDE: {analysis.get('workflow_name', '')}
{'='*80}

OVERVIEW
--------
{analysis.get('workflow_description', '')}

PURPOSE
-------
{analysis.get('workflow_purpose', '')}

{'='*80}
STEP 1: CREATE MICROSOFT FORM
{'='*80}

1. Go to Microsoft Forms (forms.microsoft.com)
2. Create new form: "{form_schema.get('title', '')}"
3. Add the following questions:

"""
    
    for idx, field in enumerate(form_schema.get('questions', []), 1):
        guide += f"""
   Question {idx}: {field.get('title', field.get('question', ''))}
   - Type: {field.get('type', '')}
   - Required: {'Yes' if field.get('required') else 'No'}
   - Validation: {field.get('validation', 'None')}
"""
    
    guide += f"""

4. Configure form settings:
   - One response per person: Yes
   - Allow anonymous responses: No
   - Confirmation message: "{form_schema.get('settings', {}).get('confirmation_message', '')}"

{'='*80}
STEP 2: CREATE EXCEL TRACKING TABLE
{'='*80}

1. Go to SharePoint: {excel_schema.get('location', '')}
2. Create new Excel file: {excel_schema.get('file_name', '')}
3. Create table: {excel_schema.get('table_name', '')}
4. Add columns:

"""
    
    for col in excel_schema.get('columns', [])[:10]:
        guide += f"   • {col['name']} ({col['type']})\n"
    
    guide += f"\n   ... and {len(excel_schema.get('columns', [])) - 10} more columns\n"
    
    guide += f"""

5. Create views:
"""
    
    for view in excel_schema.get('views', []):
        guide += f"   • {view['name']}: {view['filter']}\n"
    
    guide += f"""

{'='*80}
STEP 3: CONFIGURE POWER AUTOMATE
{'='*80}

1. Go to Power Automate (flow.microsoft.com)
2. Create new automated flow
3. Name: "{workflow.get('name', '')}"
4. Trigger: Microsoft Forms - When a new response is submitted
5. Select your form from Step 1

CONFIGURE APPROVERS
-------------------
You need to configure email addresses for the following roles:

"""
    
    for level_info in approval_chain:
        guide += f"   • {level_info['approver_role'].upper().replace(' ', '_')}_EMAIL\n"
        guide += f"     Timeout: {level_info.get('timeout_hours', 48)} hours\n"
        guide += f"     Action on rejection: {level_info['rejection_behavior']}\n\n"
    
    guide += f"""

APPROVAL FLOW LOGIC
-------------------
"""
    
    for level_info in approval_chain:
        guide += f"\nLevel {level_info['level']}: {level_info['approver_role']}\n"
        for condition in level_info.get('conditions', []):
            guide += f"   - {condition}\n"
    
    guide += f"""

BUSINESS RULES IMPLEMENTED
---------------------------
"""
    
    for idx, rule in enumerate(analysis.get('business_rules', []), 1):
        guide += f"{idx}. {rule}\n"
    
    guide += f"""

{'='*80}
STEP 4: TEST THE WORKFLOW
{'='*80}

1. Submit a test form response
2. Verify email notifications sent
3. Check Excel row created
4. Test approval chain:
   - Approve at Level 1
   - Approve at Level 2
   - Final approval at Level 3
5. Verify final notification and document generation

TEST SCENARIOS TO COVER:
------------------------
1. Full approval path (all approvers approve)
2. Rejection at Level 1
3. Rejection at Level 2
4. Timeout scenario
5. Withdrawal request

{'='*80}
STEP 5: GO LIVE
{'='*80}

1. Share form with intended users
2. Communicate approval chain to all approvers
3. Set up monitoring dashboard in Excel
4. Document escalation procedures
5. Train support staff

SUPPORT CONTACTS
----------------
Form Issues: forms-support@university.edu
Workflow Issues: automation-support@university.edu
Excel Access: sharepoint-admin@university.edu

{'='*80}
END OF IMPLEMENTATION GUIDE
{'='*80}
"""
    
    return guide

In [22]:
def should_refine_questions(state: GraphState) -> str:
    """
    Route based on validation results
    """
    if state.get("refinement_needed", False):
        iter_count = state.get("iteration_count", 0)
        if iter_count < 3:
            print("\n🔄 Refining questions based on validation feedback...")
            return "refine"
        else:
            print("\n⚠️ Max iterations reached. Proceeding with current answers...")
            return "proceed"
    return "proceed"


def route_after_sanity_check(state: GraphState) -> str:
    """
    Route based on sanity check results
    """
    sanity_check = state.get("sanity_check", {})
    regeneration_count = state.get("regeneration_count", 0)
    
    if sanity_check.get("passed", False):
        print("✅ Sanity check passed - proceeding to display")
        return "display"
    else:
        if regeneration_count >= 3:
            print("⚠️  Maximum regeneration attempts reached - proceeding to display with issues")
            return "display"
        else:
            print(f"🔄 Sanity check failed - regenerating (attempt {regeneration_count + 1}/3)")
            return "regenerate"
    """
    Determine next step after validation
    """
    return "generate_schemas" if state.get("validation", {}).get("valid") else "collect"


In [23]:
def build_workflow_graph():
    """
    Build the complete LangGraph workflow
    """
    
    graph = StateGraph(GraphState)
    
    # Add nodes
    graph.add_node("analyze_request", analyze_request)
    graph.add_node("generate_questions", generate_questions)
    graph.add_node("collect_answers", collect_answers)
    graph.add_node("validate_answers", validate_answers)
    graph.add_node("generate_form_schema", generate_form_schema)
    graph.add_node("generate_excel_schema", generate_excel_schema)
    graph.add_node("generate_workflow", generate_power_automate_workflow)
    graph.add_node("display_workflow", display_workflow)
    
    # Define edges
    graph.set_entry_point("analyze_request")
    graph.add_edge("analyze_request", "generate_questions")
    graph.add_edge("generate_questions", "collect_answers")
    graph.add_edge("collect_answers", "validate_answers")
    
    # Conditional edge after validation
    graph.add_conditional_edges(
        "validate_answers",
        route_after_validation,
        {
            "collect": "collect_answers",
            "generate_schemas": "generate_form_schema"
        }
    )
    
    graph.add_edge("generate_form_schema", "generate_excel_schema")
    graph.add_edge("generate_excel_schema", "generate_workflow")
    graph.add_edge("generate_workflow", "display_workflow")
    graph.add_edge("display_workflow", END)
    
    return graph.compile()


In [24]:
from IPython.display import Image,display
display(Image(app.get_graph().draw_mermaid_png()))

NameError: name 'app' is not defined